In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
import plotting

In [ ]:
import scipy
from iminuit import cost
from iminuit import Minuit

In [ ]:
import hist

In [ ]:
plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
df = pd.read_csv("features_CNN_1d_99987.csv")

In [ ]:
X = np.load("images_1d_99987.npy")[:,:,np.newaxis]

In [ ]:
X /= X.max()

In [ ]:
plt.imshow(X[:200,:,0])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
import tensorflow.keras.backend as K

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
K.set_image_data_format("channels_last")

In [ ]:
y =  df['start_z'].values

In [ ]:
y = (y + 235)/155

In [ ]:
def y_to_z(y):
    return (y * 155) - 235

In [ ]:
y_to_z(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=0
    )

In [ ]:
input_shape = (200, 1)

In [ ]:
from tensorflow.keras.layers import Conv1D

In [ ]:
from tensorflow.keras.layers import MaxPooling1D

In [ ]:
from tensorflow.keras import Sequential

In [ ]:
model_name = "CNN_1d_cotopaxi"

In [ ]:
model = Sequential([
    Input(input_shape),
    Conv1D(
        16,
        kernel_size=(3),
        padding="same",
        activation="elu",
    ),
    Conv1D(
        16,
        kernel_size=(3),
        padding="same",
        activation="elu",
    ),
    MaxPooling1D(2), # Pool planes in station
    Conv1D(
        16,
        kernel_size=(3),
        padding="same",
        activation="elu",
    ),
    MaxPooling1D(3),
    Dropout(0.5),
    Conv1D(
        16,
        kernel_size=(3),
        padding="same",
        activation="elu",
    ),
    Flatten(),
    Dense(1)]
)    

In [ ]:
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
plot_model(model)

In [ ]:
history_df = None

In [ ]:
fit_result = model.fit(
        x=X_train, y=y_train, batch_size=128, epochs=100
    )

In [ ]:
history_df = pd.concat([history_df, pd.DataFrame(fit_result.history)])

In [ ]:
model.save(f"{model_name}_n{len(y)}_e{len(history_df)}.keras")

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title("CNN start z")
ax1.plot(history_df["loss"].values, color=colors[0])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss Function", color=colors[0])
ax2.plot(history_df["mae"].values, color=colors[1])
ax2.set_ylabel("Error", color=colors[1])
plt.text(0.3, 0.7, f"Training dataset: {len(y_train)} events\n" f"Test dataset: {len(y_test)} events\n" f"Training duration: {len(history_df)} epochs\n{model_name}", transform=ax1.transAxes)
plt.savefig(f"plots/convergence_{model_name}_n{len(y)}_e{len(history_df)}.pdf")
plt.savefig(f"plots/convergence_{model_name}_n{len(y)}_e{len(history_df)}.png")

In [ ]:
y_pred = model.predict(x=X_test)

In [ ]:
z_pred = y_to_z(y_pred)

In [ ]:
h = hist.Hist.new.Regular(20, -5, +5, name=r"𝛥z [cm]").Double()

In [ ]:
h.fill(np.squeeze(z_pred) - np.squeeze(y_to_z(y_test)))

In [ ]:
entries, edges = h.to_numpy()

In [ ]:
def residual_model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, mu, sigma)

In [ ]:
m = Minuit(cost.BinnedNLL(entries, edges, residual_model), 0, 25)

In [ ]:
res = m.migrad()

In [ ]:
res

In [ ]:
h.plot()
plt.xlabel(r"$\Delta z\;[\mathrm{cm}]$")
plot_range = edges[0], edges[-1]
x = np.linspace(*plot_range, 100)
best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
# best_fit = scipy.stats.norm(0.044, 2.83) # TODO take from fit
n_bins = len(entries)
binsize = (plot_range[1] - plot_range[0]) / n_bins
scale = h.sum() / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0])) * binsize
plt.plot(x, scale * best_fit.pdf(x))
ax = plt.gca()
# plt.text(0.6, 0.9, r"$\mu = 0.044 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.9,
    rf"$\mu = {res.params[0].value:.2f} \pm {res.params[0].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
plt.text(0.25, 0.1, f"Training dataset: {len(y_train)} events\n" f"Test dataset: {len(y_test)} events\n" f"Training duration: {len(history_df)} epochs\n{model_name}", transform=ax.transAxes, usetex=True)
# plt.text(0.6, 0.81, r"$\sigma = 2.83 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.81,
    rf"$\sigma = {res.params[1].value:.2f} \pm {res.params[1].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
plotting.watermark()
plt.savefig(f"plots/h_dz_{model_name}_n{len(y)}_e{len(history_df)}.pdf")
plt.savefig(f"plots/h_dz_{model_name}_n{len(y)}_e{len(history_df)}.png")

In [ ]:
plt.hist(y_to_z(y_train))
plt.hist(y_to_z(y_test))